# Uncertainty Analysis

The following code takes the full list of parameters which can be calibrated and sets the uncertainty limits based off of literature found.

In [1]:
import pandas as pd
import math

A 95% confidence interval is used (Z = 1.96) to set the uncertainty limits. 

In [2]:
#95% Confidence Interval
StandardDeviationNumber = 1.96

In [3]:
#Full List of Tuneable Parameters
df = pd.read_csv('../TuneableParameters.csv')
print(len(df))

4431


In [4]:
classesToIgnore = [
    'ProgramControl', 'Schedule:Day:List', 'HeatBalanceAlgorithm', 'Sizing:Parameters', 'Sizing:Zone',
    'Daylighting:Controls', 'Site:Location', 'Building', 'DesignSpecification:ZoneAirDistribution', 'Schedule:Day:Hourly',
    'ShadowCalculation', 'ConvergenceLimits' , 'ZoneCapacitanceMultiplier:ResearchSpecial'
]


specificFieldsToIgnoreMaterial = ['Thickness']

specificFieldsToIgnoreWindowMaterialGlazing = ['Thickness', 'Dirt Correction Factor for Solar and Visible Transmittance']

specificFieldsToIgnoreWindowPropertyFrameAndDivider = [
    'Number of Horizontal Dividers', 'Number of Vertical Dividers', 'Frame Width', 'Divider Width',
    'Ratio of Frame-Edge Glass Conductance to Center-Of-Glass Conductance',
    'Ratio of Divider-Edge Glass Conductance to Center-Of-Glass Conductance'
]

specificFieldsToIgnoreZoneInfiltrationDesignFlowRate = ['Constant Term Coefficient']

In [5]:
df = df.loc[(~df['Class'].isin(classesToIgnore))]
print(len(df))

2180


In [6]:
df = df.loc[(~df['Field'].isin(specificFieldsToIgnoreMaterial))]
print(len(df))

2165


In [7]:
df = df.loc[(~df['Field'].isin(specificFieldsToIgnoreWindowMaterialGlazing))]
print(len(df))

2164


In [8]:
df = df.loc[(~df['Field'].isin(specificFieldsToIgnoreWindowPropertyFrameAndDivider))]
print(len(df))

2158


In [9]:
df = df.loc[(~df['Field'].isin(specificFieldsToIgnoreZoneInfiltrationDesignFlowRate))]
print(len(df))

2041


### Outside Environment

In [10]:
SiteGroundTemperatures = [
    'Site:GroundTemperature:BuildingSurface', 'Site:GroundTemperature:Deep', 'Site:GroundTemperature:Shallow',
    'Site:GroundTemperature:FCfactorMethod'
]

SiteGroundReflectances = [
    'Site:GroundReflectance'
]

ZoneInfiltrationDesignFlowRates = [
    'ZoneInfiltration:DesignFlowRate'
]

DesignSpecificationOutdoorAirs = [
    'DesignSpecification:OutdoorAir'
]

### Materials

In [11]:
MacDonaldsDict = {}
AbsorptivityDict = {}



MacDonaldsDict['Floor/Roof Screed_0.05'] = {'Conductivity': [0.787, 0.341, 0.077], 'Density': [1481, 390, 25],
                                       'Specific Heat': [904, 125, 93], 'Sample Size': 8}

MacDonaldsDict['Cast Concrete_0.15'] = {'Conductivity': [1.491, 0.3, 0.144], 'Density': [2179, 149, 36],
                                       'Specific Heat': [864, 92, 91], 'Sample Size': 14}

MacDonaldsDict['Cast Concrete (Dense)_0.1'] = {'Conductivity': [1.491, 0.3, 0.144], 'Density': [2179, 149, 36],
                                       'Specific Heat': [864, 92, 91], 'Sample Size': 14}

MacDonaldsDict['Cast Concrete_0.2'] = {'Conductivity': [1.491, 0.3, 0.144], 'Density': [2179, 149, 36],
                                       'Specific Heat': [864, 92, 91], 'Sample Size': 14}

MacDonaldsDict['Chipboard_0.013'] = {'Conductivity': [0.151, 0.042, 0.021], 'Density': [578, 148, 25],
                                       'Specific Heat': [2162, 646, 134], 'Sample Size': 27}

MacDonaldsDict['Standard insulation_0.0096'] = {'Conductivity': [0.039, 0.014, 0.003], 'Density': [38, 27, 3],
                                       'Specific Heat': [1072, 298, 57], 'Sample Size': 84}

MacDonaldsDict['External Rendering_0.025'] = {'Conductivity': [0.787, 0.341, 0.077], 'Density': [1481, 390, 25],
                                       'Specific Heat': [904, 125, 93], 'Sample Size': 8}

MacDonaldsDict['XPS Extruded Polystyrene  - CO2 Blowing_0.03'] = {'Conductivity': [0.054, 0.022, 0.004], 'Density': [84, 59, 4],
                                       'Specific Heat': [796, 153, 50], 'Sample Size': 124}



AbsorptivityDict['Cast Concrete_0.15'] = 0.04
AbsorptivityDict['Cast Concrete (Dense)_0.1'] = 0.04
AbsorptivityDict['Cast Concrete_0.2'] = 0.04
AbsorptivityDict['MW Stone Wool (rolls)_0.1439'] = 0.1


WindowMaterialGlazings8Percent = [
    'Front Side Solar Reflectance at Normal Incidence', 'Back Side Solar Reflectance at Normal Incidence', 
    'Front Side Visible Reflectance at Normal Incidence', 'Back Side Visible Reflectance at Normal Incidence',
    'Front Side Infrared Hemispherical Emissivity', 'Back Side Infrared Hemispherical Emissivity',
    
]


TransmittanceProperties = [
    'Solar Transmittance at Normal Incidence', 'Visible Transmittance at Normal Incidence',
]

WindowPropertyFrameAndDividerConductances = [
    'Frame Conductance', 'Divider Conductance'
]

WindowPropertyFrameAndDividerAbsorptivities = [
    'Frame Solar Absorptance', 'Frame Visible Absorptance', 'Divider Solar Absorptance', 'Divider Visible Absorptance',
    'Outside Reveal Solar Absorptance', 'Inside Sill Solar Absorptance', 'Inside Reveal Solar Absorptance'
]

WindowPropertyFrameAndDividerEmissivities = [
    'Frame Thermal Hemispherical Emissivity', 'Divider Thermal Hemispherical Emissivity'
]

In [12]:
def adjustMaximumValues(row):
################### ENVIRONMENT ############################################################################################

#Variance in Ground Temperatures Taken from Energy Plus auxillary guidelines.
    if row['Class'] in SiteGroundTemperatures:
        return row['Default'] + 2
    
#Variance in Ground Reflectances taken from Silva 2014.
    if row['Class'] in SiteGroundReflectances:
        return 0.26 
    
    
#Uncertainty valus for ZoneInfiltration:DesignFlowRates --> Design Flow Rate given by Jones 2015.
#First these uncertainty values must be scaled.
    if (row['Class'] in ZoneInfiltrationDesignFlowRates) and (row['Field'] == 'Design Flow Rate'):
        
        objectName = row['Object'].split(' ')[0]
        for index, r in df.iterrows():
            
            if (r['Class'] == 'Zone') and (r['Object'] == objectName):
                objectVolume = r['Default']
                break
        
        sigma_Q = (0.31*objectVolume)/3600
        
        value = row['Default'] + StandardDeviationNumber*(sigma_Q/math.sqrt(10200))
        assert value > 0
        return value

    
#Uncertainty valus for DesignSpecification:OutdoorAir --> Outdoor Air Flow Air Changes per Hour given by Jones 2015.    
    if row['Class'] in DesignSpecificationOutdoorAirs:
        value = row['Default'] + StandardDeviationNumber*(0.31/math.sqrt(10200))
        assert value > 0
        return value
    
############################################################################################################################

################### MATERIALS ##############################################################################################

#All Material --> Conductivity uncertainty values taken from MacDonald 2002.
#Exactly specified for some materials, for the others it is estimated to vary by 5% (Based on MacDonald 2002).
    if (row['Class'] == 'Material') and (row['Field'] == 'Conductivity'):
        if row['Object'] in MacDonaldsDict:
            materialMean = MacDonaldsDict[row['Object']]['Conductivity'][0]
            materialStdDev = MacDonaldsDict[row['Object']]['Conductivity'][1]
            materialSampleSize = MacDonaldsDict[row['Object']]['Sample Size']
            
            value = row['Default'] + StandardDeviationNumber*(materialStdDev/math.sqrt(materialSampleSize))
            assert value > 0
            return value
        
        elif row['Object'] not in MacDonaldsDict:
            value = row['Default'] + (row['Default']*0.05)
            assert value > 0
            return value

#All Material --> Density uncertainty values taken from MacDonald 2002.
#Exactly specified for some materials, for the others it is estimated to vary by 1% (Based on MacDonald 2002).
    if (row['Class'] == 'Material') and (row['Field'] == 'Density'):
        if row['Object'] in MacDonaldsDict:
            materialMean = MacDonaldsDict[row['Object']]['Density'][0]
            materialStdDev = MacDonaldsDict[row['Object']]['Density'][1]
            materialSampleSize = MacDonaldsDict[row['Object']]['Sample Size']
            
            value = row['Default'] + StandardDeviationNumber*(materialStdDev/math.sqrt(materialSampleSize))
            assert value > 0
            return value
        
        elif row['Object'] not in MacDonaldsDict:
            value = row['Default'] + (row['Default']*0.01)
            assert value > 0
            return value

#All Material --> Specific Heat uncertainty values taken from MacDonald 2002.
#Exactly specified for some materials, for the others it is estimated to vary by 12.5% (Based on MacDonald 2002).
    if (row['Class'] == 'Material') and (row['Field'] == 'Specific Heat'):
        if row['Object'] in MacDonaldsDict:
            materialMean = MacDonaldsDict[row['Object']]['Specific Heat'][0]
            materialStdDev = MacDonaldsDict[row['Object']]['Specific Heat'][1]
            materialSampleSize = MacDonaldsDict[row['Object']]['Sample Size']
            
            value = row['Default'] + StandardDeviationNumber*(materialStdDev/math.sqrt(materialSampleSize))
            assert value > 100
            return value
        
        elif row['Object'] not in MacDonaldsDict:
            value = row['Default'] + (row['Default']*0.125)
            assert value > 0 
            return value


        
#All Material --> Thermal Absorptance uncertainty values given by MacDonald 2002.
    if (row['Class'] == 'Material') and (row['Field'] == 'Thermal Absorptance'):
        value = row['Default'] + StandardDeviationNumber*0.02
        assert value < 1
        assert value > 0
        return value

#All Material --> Solar Absorptance and Material --> Visible Absorptance uncertainty values given by MacDonald 2002.
#Exactly specified for some materials, for the others it is estimated to have a standard deviation of 0.05.
    if (row['Class'] == 'Material') and (row['Field'] == 'Solar Absorptance'):
        if row['Object'] in AbsorptivityDict:
            value = row['Default'] + AbsorptivityDict[row['Object']]*StandardDeviationNumber
            assert value < 1
            assert value > 0
            return value
        
        elif row['Object'] not in AbsorptivityDict:
            value = row['Default'] + 0.05*StandardDeviationNumber
            assert value < 1
            assert value > 0
            return value
        
    if (row['Class'] == 'Material') and (row['Field'] == 'Visible Absorptance'):
        if row['Object'] in AbsorptivityDict:
            value = row['Default'] + AbsorptivityDict[row['Object']]*StandardDeviationNumber
            assert value < 1
            assert value > 0
            return value
        
        elif row['Object'] not in AbsorptivityDict:
            value = row['Default'] + 0.05*StandardDeviationNumber
            assert value < 1
            assert value > 0
            return value
        
        

#All Material:NoMass --> Thermal Resistance set to vary like Conductivity defined by MacDonald 2002.
    if (row['Class'] == 'Material:NoMass') and (row['Field'] == 'Thermal Resistance'):
        if row['Object'] == '20_RVAL_2' or row['Object'] == '19_RVAL_2':
            value = row['Default'] + row['Default']*0.05
            assert value > 0.001
            return value
        
        elif row['Object'] == 'LinearBridgingLayer':
            value = row['Default'] + row['Default']*0.05
            assert value > 0.001
            return value
        
#All Material:NoMass --> Thermal Absorptance uncertainty values given by MacDonald 2002.    
    if (row['Class'] == 'Material:NoMass') and (row['Field'] == 'Thermal Absorptance'):
        if row['Object'] == '20_RVAL_2' or row['Object'] == '19_RVAL_2':
            value = row['Default'] + 0.02*StandardDeviationNumber
            assert value < 1
            assert value > 0
            return value
        
        elif row['Object'] == 'LinearBridgingLayer':
            value = row['Default'] + row['Default']*0.05
            assert value < 1
            assert value > 0
            return value

#All Material:NoMass --> Solar Absorptance and Material --> Visible Absorptance uncertainty values given by MacDonald 2002.
    if (row['Class'] == 'Material:NoMass') and (row['Field'] == 'Solar Absorptance'):
        if row['Object'] == '20_RVAL_2' or row['Object'] == '19_RVAL_2':
            value = row['Default'] + 0.04*StandardDeviationNumber
            assert value < 1
            assert value > 0
            return value
        
        elif row['Object'] == 'LinearBridgingLayer':
            value = row['Default'] + row['Default']*0.05
            assert value < 1
            assert value > 0
            return value
        
    if (row['Class'] == 'Material:NoMass') and (row['Field'] == 'Visible Absorptance'):
        if row['Object'] == '20_RVAL_2' or row['Object'] == '19_RVAL_2':
            value = row['Default'] + 0.04*StandardDeviationNumber
            assert value < 1
            assert value > 0
            return value
        
        elif row['Object'] == 'LinearBridgingLayer':
            value = row['Default'] + row['Default']*0.05
            assert value < 1
            assert value > 0
            return value
        
        
        
#WindowMaterial:Glazing properties defined in the WindowMaterialGlazings8Percent list above are set to have a standard
#deviation of 8% based on Young-Jin 2016.
    if (row['Class'] == 'WindowMaterial:Glazing') and (row['Field'] in WindowMaterialGlazings8Percent):
        value = row['Default'] + (row['Default']*0.08)*StandardDeviationNumber
        
        if value > 1:
            value = 0.99
            
        assert value <= 1
        assert value > 0
        return value

#WindowMaterial:Glazing --> Conductivity uncertainty values based off MacDonald 2002   . 
    if (row['Class'] == 'WindowMaterial:Glazing') and (row['Field'] == 'Conductivity'):
        value = row['Default'] + StandardDeviationNumber*(0.495/math.sqrt(2))
        assert value > 0
        return value
    
#WindowMaterial:Glazing Transmittance properties need to be specified so that the reflectances + the transmittances do not
#go over 1    
    if (row['Class'] == 'WindowMaterial:Glazing') and (row['Field'] == 'Solar Transmittance at Normal Incidence'):
        return 0.925
    
    if (row['Class'] == 'WindowMaterial:Glazing') and (row['Field'] == 'Visible Transmittance at Normal Incidence'):
        return 0.919

    
#WindowProperty:FrameAndDivider --> Frame Conductance and WindowProperty:FrameAndDivider --> Divider Conductance uncertainty
#values based off MacDonald 2002.
    if (row['Class'] == 'WindowProperty:FrameAndDivider') and (row['Field'] in WindowPropertyFrameAndDividerConductances):
        value = row['Default'] + StandardDeviationNumber*(0.188/math.sqrt(3))
        assert value > 0
        return value

#WindowProperty:FrameAndDivider Absorptivity uncertainty values based off MacDonald 2002.    
    if (row['Class'] == 'WindowProperty:FrameAndDivider') and (row['Field'] in WindowPropertyFrameAndDividerAbsorptivities):
        value = row['Default'] + 0.05*StandardDeviationNumber
        assert value < 1
        assert value > 0
        return value

#WindowProperty:FrameAndDivider Emissivity uncertainty values based off MacDonald 2002.   
    if (row['Class'] == 'WindowProperty:FrameAndDivider') and (row['Field'] in WindowPropertyFrameAndDividerEmissivities):
        value = row['Default'] + 0.02*StandardDeviationNumber
        assert value > 0
        return value
    
############################################################################################################################

################### OCCUPANCY ##############################################################################################

#People --> Number of People set to have a standard deviation of 40% based off of Belazi 2018
#People --> Carbon Dioxide Generation Rate uncertainty limits based off of Persily 2017
    if (row['Class'] == 'People'):
        if row['Field'] == 'Number of People':
            value = row['Default'] + (row['Default']*0.4)*StandardDeviationNumber
            assert value > 0
            return value
        
        if row['Field'] == 'Fraction Radiant':
            value = row['Default'] + row['Default']*0.05
            assert value < 1
            assert value > 0
            return value
    
        if row['Field'] == 'Carbon Dioxide Generation Rate':
            return 0.000000044
       
    
#Lighting --> Lighting Level uncertainty limits based off of Wang 2012
    if (row['Class'] == 'Lights'):
        if row['Field'] == 'Lighting Level':
            value = row['Default'] + row['Default']*0.09
            assert value > 0
            return value
        
        if row['Field'] == 'Fraction Radiant':
            value = row['Default'] + row['Default']*0.05
            assert value < 1
            assert value > 0
            return value
        
        if row['Field'] == 'Fraction Visible':
            value = row['Default'] + row['Default']*0.05
            assert value < 1
            assert value > 0
            return value
        
        if row['Field'] == 'Fraction Replaceable':
            value = row['Default'] + row['Default']*0.05
            
            if value > 1:
                value = 1
                
            assert value <= 1
            assert value > 0
            return value
        
        
        
#ElectricEquipment --> Design Level uncertainty limits based off of Wang 2012        
    if (row['Class'] == 'ElectricEquipment'):
        if row['Field'] == 'Design Level':
            value = row['Default'] + row['Default']*0.07
            assert value > 0
            return value
        
        if row['Field'] == 'Fraction Radiant':
            value = row['Default'] + row['Default']*0.05
            assert value < 1
            assert value > 0
            return value
        
        
#WaterUse:Equipment --> Peak Flow Rate uncertainty values based off of Suchacek 2018     
    if (row['Class'] == 'WaterUse:Equipment'):
        if row['Field'] == 'Peak Flow Rate':
            value = row['Default'] + row['Default']*0.465
            assert value > 0
            return value
        
############################################################################################################################
    
    else:
        if row['Class'] != 'Zone':
            theClass = row['Class']
            theField = row['Field']
            print(f'{theClass} --> {theField} limits not specified')
            
        return 1

In [13]:
def adjustMinimumValues(row):
################### ENVIRONMENT ############################################################################################

#Variance in Ground Temperatures Taken from Energy Plus auxillary guidelines.
    if row['Class'] in SiteGroundTemperatures:
        return row['Default'] - 2
    
#Variance in Ground Reflectances taken from Silva 2014.
    if row['Class'] in SiteGroundReflectances:
        return 0.13 
    
    
#Uncertainty valus for ZoneInfiltration:DesignFlowRates --> Design Flow Rate given by Jones 2015.
#First these uncertainty values must be scaled.
    if (row['Class'] in ZoneInfiltrationDesignFlowRates) and (row['Field'] == 'Design Flow Rate'):
        
        objectName = row['Object'].split(' ')[0]
        for index, r in df.iterrows():
            
            if (r['Class'] == 'Zone') and (r['Object'] == objectName):
                objectVolume = r['Default']
                break
        
        sigma_Q = (0.31*objectVolume)/3600
        
        value = row['Default'] - StandardDeviationNumber*(sigma_Q/math.sqrt(10200))
        assert value > 0
        return value

    
#Uncertainty valus for DesignSpecification:OutdoorAir --> Outdoor Air Flow Air Changes per Hour given by Jones 2015.    
    if row['Class'] in DesignSpecificationOutdoorAirs:
        value = row['Default'] - StandardDeviationNumber*(0.31/math.sqrt(10200))
        assert value > 0
        return value
    
############################################################################################################################

################### MATERIALS ##############################################################################################

#All Material --> Conductivity uncertainty values taken from MacDonald 2002.
#Exactly specified for some materials, for the others it is estimated to vary by 5% (Based on MacDonald 2002).
    if (row['Class'] == 'Material') and (row['Field'] == 'Conductivity'):
        if row['Object'] in MacDonaldsDict:
            materialMean = MacDonaldsDict[row['Object']]['Conductivity'][0]
            materialStdDev = MacDonaldsDict[row['Object']]['Conductivity'][1]
            materialSampleSize = MacDonaldsDict[row['Object']]['Sample Size']
            
            value = row['Default'] - StandardDeviationNumber*(materialStdDev/math.sqrt(materialSampleSize))
            assert value > 0
            return value
        
        elif row['Object'] not in MacDonaldsDict:
            value = row['Default'] - (row['Default']*0.05)
            assert value > 0
            return value

#All Material --> Density uncertainty values taken from MacDonald 2002.
#Exactly specified for some materials, for the others it is estimated to vary by 1% (Based on MacDonald 2002).
    if (row['Class'] == 'Material') and (row['Field'] == 'Density'):
        if row['Object'] in MacDonaldsDict:
            materialMean = MacDonaldsDict[row['Object']]['Density'][0]
            materialStdDev = MacDonaldsDict[row['Object']]['Density'][1]
            materialSampleSize = MacDonaldsDict[row['Object']]['Sample Size']
            
            value = row['Default'] - StandardDeviationNumber*(materialStdDev/math.sqrt(materialSampleSize))
            assert value > 0
            return value
        
        elif row['Object'] not in MacDonaldsDict:
            value = row['Default'] - (row['Default']*0.01)
            assert value > 0
            return value

#All Material --> Specific Heat uncertainty values taken from MacDonald 2002.
#Exactly specified for some materials, for the others it is estimated to vary by 12.5% (Based on MacDonald 2002).
    if (row['Class'] == 'Material') and (row['Field'] == 'Specific Heat'):
        if row['Object'] in MacDonaldsDict:
            materialMean = MacDonaldsDict[row['Object']]['Specific Heat'][0]
            materialStdDev = MacDonaldsDict[row['Object']]['Specific Heat'][1]
            materialSampleSize = MacDonaldsDict[row['Object']]['Sample Size']
            
            value = row['Default'] - StandardDeviationNumber*(materialStdDev/math.sqrt(materialSampleSize))
            assert value > 100
            return value
        
        elif row['Object'] not in MacDonaldsDict:
            value = row['Default'] - (row['Default']*0.125)
            assert value > 0 
            return value


        
#All Material --> Thermal Absorptance uncertainty values given by MacDonald 2002.
    if (row['Class'] == 'Material') and (row['Field'] == 'Thermal Absorptance'):
        value = row['Default'] - StandardDeviationNumber*0.02
        assert value < 1
        assert value > 0
        return value

#All Material --> Solar Absorptance and Material --> Visible Absorptance uncertainty values given by MacDonald 2002.
#Exactly specified for some materials, for the others it is estimated to have a standard deviation of 0.05.
    if (row['Class'] == 'Material') and (row['Field'] == 'Solar Absorptance'):
        if row['Object'] in AbsorptivityDict:
            value = row['Default'] - AbsorptivityDict[row['Object']]*StandardDeviationNumber
            assert value < 1
            assert value > 0
            return value
        
        elif row['Object'] not in AbsorptivityDict:
            value = row['Default'] - 0.05*StandardDeviationNumber
            assert value < 1
            assert value > 0
            return value
        
    if (row['Class'] == 'Material') and (row['Field'] == 'Visible Absorptance'):
        if row['Object'] in AbsorptivityDict:
            value = row['Default'] - AbsorptivityDict[row['Object']]*StandardDeviationNumber
            assert value < 1
            assert value > 0
            return value
        
        elif row['Object'] not in AbsorptivityDict:
            value = row['Default'] - 0.05*StandardDeviationNumber
            assert value < 1
            assert value > 0
            return value
        
        

#All Material:NoMass --> Thermal Resistance set to vary like Conductivity defined by MacDonald 2002.
    if (row['Class'] == 'Material:NoMass') and (row['Field'] == 'Thermal Resistance'):
        if row['Object'] == '20_RVAL_2' or row['Object'] == '19_RVAL_2':
            value = row['Default'] - row['Default']*0.05
            assert value > 0.001
            return value
        
        elif row['Object'] == 'LinearBridgingLayer':
            value = row['Default'] - row['Default']*0.05
            assert value > 0.001
            return value
        
#All Material:NoMass --> Thermal Absorptance uncertainty values given by MacDonald 2002.    
    if (row['Class'] == 'Material:NoMass') and (row['Field'] == 'Thermal Absorptance'):
        if row['Object'] == '20_RVAL_2' or row['Object'] == '19_RVAL_2':
            value = row['Default'] - 0.02*StandardDeviationNumber
            assert value < 1
            assert value > 0
            return value
        
        elif row['Object'] == 'LinearBridgingLayer':
            value = row['Default'] - row['Default']*0.05
            assert value < 1
            assert value > 0
            return value

#All Material:NoMass --> Solar Absorptance and Material --> Visible Absorptance uncertainty values given by MacDonald 2002.
    if (row['Class'] == 'Material:NoMass') and (row['Field'] == 'Solar Absorptance'):
        if row['Object'] == '20_RVAL_2' or row['Object'] == '19_RVAL_2':
            value = row['Default'] - 0.04*StandardDeviationNumber
            assert value < 1
            assert value > 0
            return value
        
        elif row['Object'] == 'LinearBridgingLayer':
            value = row['Default'] - row['Default']*0.05
            assert value < 1
            assert value > 0
            return value
        
    if (row['Class'] == 'Material:NoMass') and (row['Field'] == 'Visible Absorptance'):
        if row['Object'] == '20_RVAL_2' or row['Object'] == '19_RVAL_2':
            value = row['Default'] - 0.04*StandardDeviationNumber
            assert value < 1
            assert value > 0
            return value
        
        elif row['Object'] == 'LinearBridgingLayer':
            value = row['Default'] - row['Default']*0.05
            assert value < 1
            assert value > 0
            return value
        
        
        
#WindowMaterial:Glazing properties defined in the WindowMaterialGlazings8Percent list above are set to have a standard
#deviation of 8% based on Young-Jin 2016.
    if (row['Class'] == 'WindowMaterial:Glazing') and (row['Field'] in WindowMaterialGlazings8Percent):
        value = row['Default'] - (row['Default']*0.08)*StandardDeviationNumber
        
        if value > 1:
            value = 1
            
        assert value <= 1
        assert value > 0
        return value

#WindowMaterial:Glazing --> Conductivity uncertainty values based off MacDonald 2002   . 
    if (row['Class'] == 'WindowMaterial:Glazing') and (row['Field'] == 'Conductivity'):
        value = row['Default'] - StandardDeviationNumber*(0.495/math.sqrt(2))
        assert value > 0
        return value
    
#WindowMaterial:Glazing Transmittance properties need to be specified so that the reflectances + the transmittances do not
#go over 1
    if (row['Class'] == 'WindowMaterial:Glazing') and (row['Field'] == 'Solar Transmittance at Normal Incidence'):
        return row['Default'] - (row['Default']*0.08)*StandardDeviationNumber
    
    if (row['Class'] == 'WindowMaterial:Glazing') and (row['Field'] == 'Visible Transmittance at Normal Incidence'):
        return row['Default'] - (row['Default']*0.08)*StandardDeviationNumber   

    
#WindowProperty:FrameAndDivider --> Frame Conductance and WindowProperty:FrameAndDivider --> Divider Conductance uncertainty
#values based off MacDonald 2002.
    if (row['Class'] == 'WindowProperty:FrameAndDivider') and (row['Field'] in WindowPropertyFrameAndDividerConductances):
        value = row['Default'] - StandardDeviationNumber*(0.188/math.sqrt(3))
        assert value > 0
        return value

#WindowProperty:FrameAndDivider Absorptivity uncertainty values based off MacDonald 2002.    
    if (row['Class'] == 'WindowProperty:FrameAndDivider') and (row['Field'] in WindowPropertyFrameAndDividerAbsorptivities):
        value = row['Default'] - 0.05*StandardDeviationNumber
        assert value < 1
        assert value > 0
        return value

#WindowProperty:FrameAndDivider Emissivity uncertainty values based off MacDonald 2002.   
    if (row['Class'] == 'WindowProperty:FrameAndDivider') and (row['Field'] in WindowPropertyFrameAndDividerEmissivities):
        value = row['Default'] - 0.02*StandardDeviationNumber
        assert value > 0
        return value
    
############################################################################################################################

################### OCCUPANCY ##############################################################################################

#People --> Number of People set to have a standard deviation of 40% based off of Belazi 2018
#People --> Carbon Dioxide Generation Rate uncertainty limits based off of Persily 2017
    if (row['Class'] == 'People'):
        if row['Field'] == 'Number of People':
            value = row['Default'] - (row['Default']*0.4)*StandardDeviationNumber
            assert value > 0
            return value
        
        if row['Field'] == 'Fraction Radiant':
            value = row['Default'] - row['Default']*0.05
            assert value < 1
            assert value > 0
            return value
    
        if row['Field'] == 'Carbon Dioxide Generation Rate':
            return 0.000000033
       
    
#Lighting --> Lighting Level uncertainty limits based off of Wang 2012
    if (row['Class'] == 'Lights'):
        if row['Field'] == 'Lighting Level':
            value = row['Default'] - row['Default']*0.058
            assert value > 0
            return value
        
        if row['Field'] == 'Fraction Radiant':
            value = row['Default'] - row['Default']*0.05
            assert value < 1
            assert value > 0
            return value
        
        if row['Field'] == 'Fraction Visible':
            value = row['Default'] - row['Default']*0.05
            assert value < 1
            assert value > 0
            return value
        
        if row['Field'] == 'Fraction Replaceable':
            value = row['Default'] - row['Default']*0.05
            
            if value > 1:
                value = 1
                
            assert value <= 1
            assert value > 0
            return value
        
        
        
#ElectricEquipment --> Design Level uncertainty limits based off of Wang 2012        
    if (row['Class'] == 'ElectricEquipment'):
        if row['Field'] == 'Design Level':
            value = row['Default'] - row['Default']*0.113
            assert value > 0
            return value
        
        if row['Field'] == 'Fraction Radiant':
            value = row['Default'] - row['Default']*0.05
            assert value < 1
            assert value > 0
            return value
        
        
#WaterUse:Equipment --> Peak Flow Rate uncertainty values based off of Suchacek 2018     
    if (row['Class'] == 'WaterUse:Equipment'):
        if row['Field'] == 'Peak Flow Rate':
            value = row['Default'] - row['Default']*0.318
            assert value > 0
            return value
        
############################################################################################################################
    
    else:
        if row['Class'] != 'Zone':
            theClass = row['Class']
            theField = row['Field']
            print(f'{theClass} --> {theField} limits not specified')
            
        return 1

In [14]:
df['Maximum'] = df.apply(lambda row: adjustMaximumValues(row), axis=1)
df['Minimum'] = df.apply(lambda row: adjustMinimumValues(row), axis=1)

In [15]:
solarTransIndex = df.loc[df['Field'] == 'Solar Transmittance at Normal Incidence', 'Maximum'].index[0]
solarTransValue = df.loc[df['Field'] == 'Solar Transmittance at Normal Incidence', 'Maximum'][solarTransIndex]

FS_SolarReflectanceIndex = df.loc[df['Field'] == 'Front Side Solar Reflectance at Normal Incidence', 'Default'].index[0]
FS_SolarReflectanceValue = df.loc[df['Field'] == 'Front Side Solar Reflectance at Normal Incidence', 'Default'][FS_SolarReflectanceIndex]

BS_SolarReflectanceIndex = df.loc[df['Field'] == 'Back Side Solar Reflectance at Normal Incidence', 'Default'].index[0]
BS_SolarReflectanceValue = df.loc[df['Field'] == 'Back Side Solar Reflectance at Normal Incidence', 'Default'][BS_SolarReflectanceIndex]


assert solarTransValue + FS_SolarReflectanceValue <= 1
assert solarTransValue + BS_SolarReflectanceValue <= 1

In [16]:
visibleTransIndex = df.loc[df['Field'] == 'Visible Transmittance at Normal Incidence', 'Maximum'].index[0]
visibleTransValue = df.loc[df['Field'] == 'Visible Transmittance at Normal Incidence', 'Maximum'][visibleTransIndex]

FS_VisibleReflectanceIndex = df.loc[df['Field'] == 'Front Side Visible Reflectance at Normal Incidence', 'Default'].index[0]
FS_VisibleReflectanceValue = df.loc[df['Field'] == 'Front Side Visible Reflectance at Normal Incidence', 'Default'][FS_VisibleReflectanceIndex]

BS_VisibleReflectanceIndex = df.loc[df['Field'] == 'Back Side Visible Reflectance at Normal Incidence', 'Default'].index[0]
BS_VisibleReflectanceValue = df.loc[df['Field'] == 'Back Side Visible Reflectance at Normal Incidence', 'Default'][BS_VisibleReflectanceIndex]


assert visibleTransValue + FS_VisibleReflectanceValue <= 1
assert visibleTransValue + BS_VisibleReflectanceValue <= 1

In [17]:
df = df.loc[df['Class'] != 'Zone']
print(len(df))

1573


In [18]:
makeCSV = False

if makeCSV:
    df.to_csv('Filtered_TuneableParameters.csv')